In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np

from stable_baselines import PPO1
from stable_baselines.common.policies import MlpPolicy

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            return 500 + 500 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            
            if self.env.steps_in_run > 1:
                
                reward += -1
                
                # Get distance moved towards target in last step
                target_diff = self.env.distances[-2] - self.env.distances[-1]
                
                # Calculate total distance moved in last step
                gps_diff_0 = self.env.history[-2].gps_actual[0] - self.env.history[-1].gps_actual[0]
                gps_diff_1 = self.env.history[-2].gps_actual[1] - self.env.history[-1].gps_actual[1]
                actual_diff = np.sqrt(gps_diff_0**2 + gps_diff_1**2)
                
                if actual_diff > 0:
                    diff_rew = abs(target_diff / env.distances[0] * 500) * (target_diff / actual_diff)
                else:
                    diff_rew = 0
               
                reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -1
        
            if self.env.state.touching:
                reward += -20
        return reward

    def check_done(self):
        if self.env.total_reward < -1000:
            return True
        
        if self.env.steps_in_run > 500:
            return True
        
        if self.env.get_target_distance(False) < 0.1:
            return True
        
        if self.env.total_reward > 2000:
            return True
        
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 6
config.num_obstacles = 4
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 5
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
                action_class = wg.WbtActDiscrete(config=config, 
                                                 range_direction=0.1, 
                                                 range_speed=0.1, 
                                                 speeds=3, 
                                                 directions=3),
                config=config)

../../backend/webotsgym/env/webotenv.py:123: UserWarning: Relative property of action class is overwritten by config.relative_action. This might interfere with bounds argument for WbtActContinuous.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
model_name = "6x6_const_seed" # seed has to be set manually backend
model = PPO1("MlpPolicy", 
             env, 
             timesteps_per_actorbatch=2000, 
             tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=100000)
    model.save("./model_{}".format(model_name))



Calc_reward target bonus:  829.0744125843048
Calc_reward target bonus:  606.3292622566223
Calc_reward target bonus:  973.9390164613724
Calc_reward target bonus:  791.990116238594
Calc_reward target bonus:  848.6204594373703
Calc_reward target bonus:  713.0983471870422
Calc_reward target bonus:  753.9809048175812
Calc_reward target bonus:  945.7940459251404
Calc_reward target bonus:  855.7758331298828
Calc_reward target bonus:  639.5816504955292
Calc_reward target bonus:  552.9631972312927
Calc_reward target bonus:  866.4675801992416
Calc_reward target bonus:  541.9920384883881
Calc_reward target bonus:  906.0433506965637
Calc_reward target bonus:  674.5168566703796
Calc_reward target bonus:  516.431987285614
Calc_reward target bonus:  726.0707318782806
Calc_reward target bonus:  756.4390003681183
Calc_reward target bonus:  913.7007892131805
Calc_reward target bonus:  515.7147347927094
Calc_reward target bonus:  627.9441714286804
Calc_reward target bonus:  956.3236124813557
Calc_rewar

Calc_reward target bonus:  533.4038436412811
Calc_reward target bonus:  494.66896057128906
Calc_reward target bonus:  564.0288591384888
Calc_reward target bonus:  515.0497257709503
Calc_reward target bonus:  601.7595827579498
Calc_reward target bonus:  668.5215532779694
Calc_reward target bonus:  498.19451570510864
Calc_reward target bonus:  515.0642096996307
Calc_reward target bonus:  597.8296995162964
Calc_reward target bonus:  628.6289691925049
Calc_reward target bonus:  495.78362703323364
Calc_reward target bonus:  896.6325744986534
Calc_reward target bonus:  758.1900507211685
Calc_reward target bonus:  574.3338763713837
Calc_reward target bonus:  543.5649454593658
Calc_reward target bonus:  524.7042775154114
Calc_reward target bonus:  536.4452600479126
Calc_reward target bonus:  526.3029932975769
Calc_reward target bonus:  481.13930225372314
Calc_reward target bonus:  852.2098958492279
Calc_reward target bonus:  883.5419565439224
Calc_reward target bonus:  511.35319471359253
Calc_

Calc_reward target bonus:  478.48713397979736
Calc_reward target bonus:  520.4788446426392
Calc_reward target bonus:  664.0816330909729
Calc_reward target bonus:  749.6337294578552
Calc_reward target bonus:  825.501099228859
Calc_reward target bonus:  543.0384278297424
Calc_reward target bonus:  610.413670539856
Calc_reward target bonus:  570.9325075149536
Calc_reward target bonus:  537.8150939941406
Calc_reward target bonus:  588.1708264350891
Calc_reward target bonus:  694.1534578800201
Calc_reward target bonus:  570.0784921646118
Calc_reward target bonus:  602.5483906269073
Calc_reward target bonus:  594.8390662670135
Calc_reward target bonus:  770.7865387201309
Calc_reward target bonus:  481.18627071380615
Calc_reward target bonus:  534.3085527420044
Calc_reward target bonus:  532.2761237621307
Calc_reward target bonus:  546.7654466629028
Calc_reward target bonus:  561.4719390869141
Calc_reward target bonus:  608.2599461078644
Calc_reward target bonus:  760.3173851966858
Calc_rewar

Calc_reward target bonus:  573.2386410236359
Calc_reward target bonus:  553.2988011837006
Calc_reward target bonus:  952.9177285730839
Calc_reward target bonus:  648.0496823787689
Calc_reward target bonus:  768.849790096283
Calc_reward target bonus:  494.9692487716675
Calc_reward target bonus:  626.084715127945
Calc_reward target bonus:  572.3292529582977
Calc_reward target bonus:  515.5188143253326
Calc_reward target bonus:  777.7925431728363
Calc_reward target bonus:  752.9144734144211
Calc_reward target bonus:  923.4434068202972
Calc_reward target bonus:  881.3170939683914
Calc_reward target bonus:  550.5830347537994
Calc_reward target bonus:  792.0041382312775
Calc_reward target bonus:  765.2353942394257
Calc_reward target bonus:  575.6202340126038
Calc_reward target bonus:  539.8572385311127
Calc_reward target bonus:  694.3290829658508
Calc_reward target bonus:  734.1314554214478
Calc_reward target bonus:  676.022857427597
Calc_reward target bonus:  585.0989520549774
Calc_reward t

Calc_reward target bonus:  510.7973515987396
Calc_reward target bonus:  537.3537838459015
Calc_reward target bonus:  615.3248846530914
Calc_reward target bonus:  680.9675991535187
Calc_reward target bonus:  613.6305928230286
Calc_reward target bonus:  682.0587813854218
Calc_reward target bonus:  594.4332182407379
Calc_reward target bonus:  551.8081784248352
Calc_reward target bonus:  693.5401856899261
Calc_reward target bonus:  621.1192011833191
Calc_reward target bonus:  933.2509562373161
Calc_reward target bonus:  765.0776654481888
Calc_reward target bonus:  827.2388875484467
Calc_reward target bonus:  597.3668098449707
Calc_reward target bonus:  591.3430154323578
Calc_reward target bonus:  580.1747739315033
Calc_reward target bonus:  643.0123150348663
Calc_reward target bonus:  528.3395648002625
Calc_reward target bonus:  604.4712364673615
Calc_reward target bonus:  934.8267763853073
Calc_reward target bonus:  772.845908999443
Calc_reward target bonus:  572.4494457244873
Calc_reward

...

Calc_reward target bonus:  884.2034339904785
Calc_reward target bonus:  768.5939520597458
Calc_reward target bonus:  801.8130213022232
Calc_reward target bonus:  916.6587665677071
Calc_reward target bonus:  853.2213419675827
Calc_reward target bonus:  874.8442679643631
Calc_reward target bonus:  898.7341821193695
Calc_reward target bonus:  976.7739772796631
Calc_reward target bonus:  942.4493908882141
Calc_reward target bonus:  866.2975877523422
Calc_reward target bonus:  885.0688189268112
Calc_reward target bonus:  934.3998953700066
Calc_reward target bonus:  937.9321411252022
Calc_reward target bonus:  914.8004576563835
Calc_reward target bonus:  874.9730437994003
Calc_reward target bonus:  794.6335822343826
Calc_reward target bonus:  864.2334192991257
Calc_reward target bonus:  913.6170968413353
Calc_reward target bonus:  737.2048795223236
Calc_reward target bonus:  912.9493460059166
Calc_reward target bonus:  939.9438872933388
Calc_reward target bonus:  795.3978478908539
Calc_rewar

Calc_reward target bonus:  827.6183158159256
Calc_reward target bonus:  916.9849678874016
Calc_reward target bonus:  913.6855602264404
Calc_reward target bonus:  809.3661367893219
Calc_reward target bonus:  905.3259342908859
Calc_reward target bonus:  920.6801205873489
Calc_reward target bonus:  866.2577122449875
Calc_reward target bonus:  909.1264083981514
Calc_reward target bonus:  906.5356329083443
Calc_reward target bonus:  831.1102539300919
Calc_reward target bonus:  959.2480137944221
Calc_reward target bonus:  958.3969376981258
Calc_reward target bonus:  929.8842400312424
Calc_reward target bonus:  933.3588406443596
Calc_reward target bonus:  854.556068778038
Calc_reward target bonus:  854.9875468015671
Calc_reward target bonus:  869.3856447935104
Calc_reward target bonus:  947.5638158619404
Calc_reward target bonus:  870.7963675260544
Calc_reward target bonus:  962.4815098941326
Calc_reward target bonus:  930.5301308631897
Calc_reward target bonus:  973.3601938933134
Calc_reward

Calc_reward target bonus:  881.1560049653053
Calc_reward target bonus:  846.0693657398224
Calc_reward target bonus:  841.3102924823761
Calc_reward target bonus:  941.8564401566982
Calc_reward target bonus:  870.5351501703262
Calc_reward target bonus:  964.768823236227
Calc_reward target bonus:  857.4705570936203
Calc_reward target bonus:  910.0290462374687
Calc_reward target bonus:  890.851154923439
Calc_reward target bonus:  811.1090660095215
Calc_reward target bonus:  857.1551293134689
Calc_reward target bonus:  922.8742942214012
Calc_reward target bonus:  917.1818718314171
Calc_reward target bonus:  904.957227408886
Calc_reward target bonus:  928.6431670188904
Calc_reward target bonus:  820.5877244472504
Calc_reward target bonus:  828.4613937139511
Calc_reward target bonus:  826.7989009618759
Calc_reward target bonus:  774.3455469608307
Calc_reward target bonus:  789.1696393489838
Calc_reward target bonus:  836.4264518022537
Calc_reward target bonus:  924.8808026313782
Calc_reward t

Calc_reward target bonus:  899.8075798153877
Calc_reward target bonus:  810.6357604265213
Calc_reward target bonus:  964.1238264739513
Calc_reward target bonus:  860.9934747219086
Calc_reward target bonus:  859.3062609434128
Calc_reward target bonus:  886.6069912910461
Calc_reward target bonus:  855.1843613386154
Calc_reward target bonus:  857.152059674263
Calc_reward target bonus:  924.1144806146622
Calc_reward target bonus:  888.605572283268
Calc_reward target bonus:  818.4532225131989
Calc_reward target bonus:  807.1817010641098
Calc_reward target bonus:  933.0669045448303
Calc_reward target bonus:  756.3320994377136
Calc_reward target bonus:  922.5554540753365
Calc_reward target bonus:  823.2826441526413
Calc_reward target bonus:  910.3196337819099
Calc_reward target bonus:  876.1168569326401
Calc_reward target bonus:  857.3533445596695
Calc_reward target bonus:  821.4185684919357
Calc_reward target bonus:  926.9891902804375
Calc_reward target bonus:  721.897304058075
Calc_reward t

Calc_reward target bonus:  962.2352831065655
Calc_reward target bonus:  812.4983161687851
Calc_reward target bonus:  838.3374363183975
Calc_reward target bonus:  747.1533417701721
Calc_reward target bonus:  841.3335978984833
Calc_reward target bonus:  910.9643623232841
Calc_reward target bonus:  910.0100472569466
Calc_reward target bonus:  868.8099980354309
Calc_reward target bonus:  895.4191282391548
Calc_reward target bonus:  900.8381962776184
Calc_reward target bonus:  940.4087774455547
Calc_reward target bonus:  879.9801245331764
Calc_reward target bonus:  860.9914034605026
Calc_reward target bonus:  811.9122087955475
Calc_reward target bonus:  826.6812115907669
Calc_reward target bonus:  842.9235965013504
Calc_reward target bonus:  922.2395569086075
Calc_reward target bonus:  919.3816632032394
Calc_reward target bonus:  904.6782404184341
Calc_reward target bonus:  824.7523605823517
Calc_reward target bonus:  787.6461148262024
Calc_reward target bonus:  928.3914715051651
Calc_rewar